In [1]:
import sys
sys.path.append('../')
from utils import *
from model import *
from collections import defaultdict
from gensim.parsing.preprocessing import strip_punctuation
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
data_dir = '/datadrive/data_cs/'
batch_size = 128
batch_num  = 128
epoch_num  = 200

device = torch.device("cuda:1")
graph = dill.load(open(data_dir + 'graph.pk', 'rb'))

In [3]:
train_range = {t: True for t in graph.times if t != None and t < 2015}
valid_range = {t: True for t in graph.times if t != None and t >= 2015  and t <= 2016}
test_range  = {t: True for t in graph.times if t != None and t > 2016}

In [5]:
graph.node_feature['paper'].loc[np.array([1,3,5]), 'title']

1         sender reputation in a large webmail service
3                 learning at low false positive rates
5    lexical url analysis for discriminating phishi...
Name: title, dtype: object

In [6]:
graph.node_feature['paper']

,citation,emb,id,time,title,type
0,11,"[0.25642919540405273, -1.2023894786834717, 0.0...",1598571137,2005,spam deobfuscation using a hidden markov model,paper
1,16,"[-0.29882562160491943, -1.4153543710708618, -1...",6401868,2006,sender reputation in a large webmail service,paper
2,9,"[-0.3127864897251129, -1.4695709943771362, -0....",2045950984,2011,collective classification for packed executabl...,paper
3,13,"[-0.6104609370231628, 0.0553511343896389, -1.0...",2143537651,2006,learning at low false positive rates,paper
4,11,"[0.1757459193468094, -0.35146409273147583, -0....",31745253,2005,spamato an extendable spam filter system,paper
5,3,"[-0.27183398604393005, -0.463623970746994, -0....",2007878250,2011,lexical url analysis for discriminating phishi...,paper
6,7,"[0.06311815232038498, 0.07333706319332123, -0....",1521863871,2007,evaluation of the highest probability svm near...,paper
7,12,"[0.3009788990020752, -0.44888800382614136, 0.1...",176125184,2006,online discriminative spam filter training,paper
8,8,"[-0.14155685901641846, -1.2402852773666382, 0....",133742671,2004,a multifaceted approach to spam reduction,paper
9,6,"[-0.2155977487564087, -1.025209903717041, 0.07...",2028195579,2011,a study of feature subset evaluators and featu...,paper


In [4]:
class GNN(nn.Module):
    def __init__(self, in_dim, n_hid, num_types, num_relations, n_heads, n_layers, dropout = 0.3):
        super(GNN, self).__init__()
        self.gcs = nn.ModuleList()
        self.num_types = num_types
        self.in_dim    = in_dim
        self.n_hid     = n_hid
        self.aggregat_ws   = nn.ModuleList()
        self.drop          = nn.Dropout(dropout)
        for t in range(num_types):
            self.aggregat_ws.append(nn.Linear(in_dim, n_hid))
        for l in range(n_layers):
            self.gcs.append(RAGCNConv(n_hid, n_hid, num_types, num_relations, n_heads, dropout))
    def set_device(self, device):
        self.device = device
        for gc in self.gcs:
            gc.device = device
    def forward(self, node_feature, node_type, edge_time, edge_index, edge_type):
        res = torch.zeros(node_feature.size(0), self.n_hid).to(node_feature.device)
        for t_id in range(self.num_types):
            aggregat_w = self.aggregat_ws[t_id]
            idx = (node_type == t_id)
            if idx.sum() == 0:
                continue
            res[idx] = torch.tanh(aggregat_w(node_feature[idx]))
        meta_xs = self.drop(res)
        del res
        for gc in self.gcs:
            meta_xs = gc(meta_xs, node_type, edge_index, edge_type, edge_time)
        return meta_xs

In [5]:
def gen_sample(seed, t_range, sampled_depth = 5, sampled_number = 100, batch_size = batch_size):
    np.random.seed(seed)
    feature, _time, edge_list, _, texts = sample_subgraph(graph, t_range, inp = None, \
                        sampled_depth = sampled_depth, sampled_number = sampled_number)  
    paper_ids = np.random.choice(len(_time['paper']), batch_size, replace = False)
    
    for target_type in edge_list:
        tes = edge_list[target_type]['paper']
        for relation_type in tes:
            reml = []
            for target_ser, source_ser in tes[relation_type]:
                if source_ser not in paper_ids:
                    reml += [[target_ser, source_ser]]
            edge_list[target_type]['paper'][relation_type] = reml
    return to_torch(feature, _time, edge_list, graph), paper_ids, texts[paper_ids]

def prepare_data(pool, process_ids):
    jobs = []
    for process_id in process_ids[:-1]:
        p = pool.apply_async(gen_sample, args=(np.random.randint(2**32 - 1), train_range))
        jobs.append(p)
    p = pool.apply_async(gen_sample, args=(np.random.randint(2**32 - 1), valid_range))
    jobs.append(p)
    p = pool.apply_async(gen_sample, args=(np.random.randint(2**32 - 1), valid_range))
    jobs.append(p)
    return jobs

In [6]:
class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""
    def __init__(self, n_word, ninp, nhid, nlayers, dropout=0.3):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.rnn = nn.LSTM(nhid, nhid, nlayers)
        self.encoder = nn.Embedding(n_word, nhid)
        self.decoder = nn.Linear(nhid, n_word)
        self.decoder.weight = self.encoder.weight
        self.adp     = nn.Linear(ninp + nhid, nhid)
        self.encoder.weight.require_grad = False
    def forward(self, inp, hidden = None):
        emb = self.drop(self.encoder(inp))
        if hidden is not None:
            emb = torch.cat((emb, hidden), dim=-1)
            emb = F.elu(self.adp(emb))
        output, _ = self.rnn(self.drop(emb))
        decoded = self.decoder(self.drop(output))
        return decoded
    def from_w2v(self, w2v):
        initrange = 0.1
        self.encoder.weight.data = w2v

In [7]:
edge_dict = {e[2]: i for i, e in enumerate(graph.get_meta_graph())}

In [8]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class TransformerModel(nn.Module):
    """Container module with an encoder, a recurrent or transformer module, and a decoder."""

    def __init__(self, n_word, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        try:
            from torch.nn import TransformerEncoder, TransformerEncoderLayer
        except:
            raise ImportError('TransformerEncoder module does not exist in PyTorch 1.1 or lower.')
        self.model_type = 'Transformer'
        encoder_layers = TransformerEncoderLayer(nhid, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.adp     = nn.Linear(ninp + nhid, nhid)
        self.encoder = nn.Embedding(n_word, ninp)
        self.pos_encoder = PositionalEncoding(nhid, dropout)
        self.decoder = nn.Linear(nhid, n_word)
        self.encoder.weight.require_grad = False
        self.init_weights()
        self.drop = nn.Dropout(0.3)
        self.nhid = nhid
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask
    def from_w2v(self, w2v):
        initrange = 0.1
        self.encoder.weight.data = w2v
    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, src, hidden = None):
        device = src.device
        mask = self._generate_square_subsequent_mask(len(src)).to(device)

        emb = self.encoder(src)
        if hidden is not None:
            emb = self.drop(torch.cat((emb, hidden), dim=-1))
            emb = F.elu(self.adp(emb))
        emb = self.pos_encoder(emb * math.sqrt(self.nhid))
        output = self.transformer_encoder(emb, mask)
        return self.decoder(output)

In [ ]:
types = graph.get_types()
gnn = GNN(in_dim = len(graph.node_feature['paper']['emb'][0]) + 401, n_hid = 256, num_types = len(types), \
          num_relations = len(graph.get_meta_graph()) + 1, n_heads = 8, n_layers = 4).to(device)

w2v_model = Word2Vec.load(data_dir + "word2vec.model")
n_tokens = len(w2v_model.wv.vocab)
rnn = TransformerModel(n_word = n_tokens, ninp = gnn.n_hid, nhead = 4,\
               nhid = w2v_model.vector_size, nlayers = 2).to(device)
rnn.from_w2v(torch.FloatTensor(w2v_model.wv.vectors).to(device))
optimizer = torch.optim.AdamW(list(gnn.parameters()) + list(rnn.parameters()))
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1000, eta_min=1e-6)

/home/ziniu/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# types = graph.get_types()
# gnn = GNN(in_dim = len(graph.node_feature['paper']['emb'][0]) + 401, n_hid = 256, num_types = len(types), \
#           num_relations = len(graph.get_meta_graph()) + 1, n_heads = 8, n_layers = 3).to(device)

# w2v_model = Word2Vec.load(data_dir + "word2vec.model")
# n_tokens = len(w2v_model.wv.vocab)
# rnn = RNNModel(n_word = n_tokens, ninp = gnn.n_hid, \
#                nhid = w2v_model.vector_size, nlayers = 2).to(device)
# rnn.from_w2v(torch.FloatTensor(w2v_model.wv.vectors).to(device))
# optimizer = torch.optim.AdamW(list(gnn.parameters()) + list(rnn.parameters()))
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1000, eta_min=1e-6)

In [ ]:
def parse_text(texts, w2v_model, device):
    idxs = []
    pad  = w2v_model.wv.vocab['<eos>'].index
    for text in texts:
        idx = []
        for word in ['<bos>'] + strip_punctuation(text.lower()).split() + ['<eos>']:
            if word in w2v_model.wv.vocab:
                idx += [w2v_model.wv.vocab[word].index]
        idxs += [idx]
    mxl = np.max([len(s) for s in idxs]) + 1
    inp_idxs = []
    out_idxs = []
    masks    = []
    lens     = []
    for i, idx in enumerate(idxs):
        inp_idxs += [idx + [pad for _ in range(mxl - len(idx) - 1)]]
        out_idxs += [idx[1:] + [pad for _ in range(mxl - len(idx))]]
        masks    += [[1 for _ in range(len(idx))] + [0 for _ in range(mxl - len(idx) - 1)]]
        lens     += [len(idx)]
    return torch.LongTensor(inp_idxs).transpose(0, 1).to(device), \
           torch.LongTensor(out_idxs).transpose(0, 1).to(device), torch.BoolTensor(masks).transpose(0, 1).to(device), lens

In [ ]:
stats = []
pool = mp.Pool(8)
process_ids = np.arange(batch_num // 8)
st = time.time()
jobs = prepare_data(pool, process_ids)
train_step = 1500
best_val   = 100000
criterion = nn.CrossEntropyLoss(reduction = 'none')

In [13]:
train_data = [job.get() for job in jobs[:-2]]
valid_data = [jobs[-2].get(), jobs[-1].get()]

In [14]:
data, paper_ids, texts = train_data[0]

In [15]:
node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = data

In [ ]:
for epoch in np.arange(epoch_num * 100)+1:
    train_data = [job.get() for job in jobs[:-2]]
    valid_data = [jobs[-2].get(), jobs[-1].get()]
    pool.close()
    pool.join()
    pool = mp.Pool(8)
    jobs = prepare_data(pool, process_ids)
    et = time.time()
    print('Data Preparation: %.1fs' % (et - st))
    
    train_losses = []
    gnn.train()
    rnn.train()
    torch.cuda.empty_cache()
    for data, paper_ids, texts in train_data:
        node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = data
        reps = gnn(node_feature.to(device), node_type.to(device), edge_time.to(device), \
                   edge_index.to(device), edge_type.to(device))[node_dict['paper'][0] + paper_ids]
        inp_idxs, out_idxs, masks, lens = parse_text(texts, w2v_model, device)
        pred_prob = rnn(inp_idxs, reps.repeat(inp_idxs.shape[0], 1, 1))
        loss = criterion(pred_prob[masks], out_idxs[masks]).mean()

        optimizer.zero_grad() 
        loss.backward()
        torch.nn.utils.clip_grad_norm_(rnn.parameters(), 0.2)
        optimizer.step()
        train_losses += [loss.cpu().detach().tolist()]
        train_step += 1
        scheduler.step(train_step)
        del loss, pred_prob
        torch.cuda.empty_cache()
    '''
        Valid
    '''
    gnn.eval()
    rnn.eval()
    with torch.no_grad():
        valid_accs   = []
        valid_losses = []
        for data, paper_ids, texts in valid_data:
            node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = data
            reps = gnn(node_feature.to(device), node_type.to(device), edge_time.to(device), \
                       edge_index.to(device), edge_type.to(device))[node_dict['paper'][0] + paper_ids]
            inp_idxs, out_idxs, masks, lens = parse_text(texts, w2v_model, device)
            pred_prob = rnn(inp_idxs, reps.repeat(inp_idxs.shape[0], 1, 1))
            loss = criterion(pred_prob[masks], out_idxs[masks])
            valid_loss = loss.cpu().detach().tolist()
            
            if epoch % 100 == 0:
                l_s = []
                s = 0
                for i in lens:
                    l_s += [np.average(valid_loss[s: s+i])]
                    s = s+i
                predict_sent = []
                label_sent   = []
                for s, text in zip(pred_prob.transpose(0,1).argmax(dim=-1), texts):
                    sent = []
                    for w in s:
                        word = w2v_model.wv.index2word[w]
                        sent += [w2v_model.wv.index2word[w]]
                        if word == '<eos>':
                            break
                    label_sent += [text]
                    predict_sent += [' '.join(sent)]

                for i in np.argsort(l_s):
                    print('Loss: %.4f' % l_s[i])
                    print('Predict: %s' % predict_sent[i])
                    print('Label: %s' % label_sent[i])
    
            valid_losses += [np.average(valid_loss)]
            del loss, pred_prob
        st = time.time()
        print(("Epoch: %d (%.1fs)  LR: %.5f Train Loss: %f  Valid Loss: %f") % \
              (epoch, (st-et), optimizer.param_groups[0]['lr'], np.average(train_losses), \
               np.average(valid_losses)))
        if np.average(valid_losses) < best_val:
            print('update!!!!!')
            best_val = np.average(valid_losses)
            torch.save(rnn, './save/rnn.pt')
            torch.save(gnn, './save/gen_gnn.pt')
        stats += [[train_losses, valid_losses]]

Data Preparation: 16.0s
Epoch: 1 (29.4s)  LR: 0.00052 Train Loss: 9.340126  Valid Loss: 8.566558
update!!!!!


/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TransformerModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TransformerEncoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleList. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TransformerEncoderLayer. It won't be checked for correctness upon loadin

Data Preparation: 3.7s
Epoch: 2 (28.1s)  LR: 0.00055 Train Loss: 8.302674  Valid Loss: 7.687462
update!!!!!


/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TransformerModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TransformerEncoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleList. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TransformerEncoderLayer. It won't be checked for correctness upon loadin

Data Preparation: 3.8s
Epoch: 3 (27.2s)  LR: 0.00057 Train Loss: 7.359079  Valid Loss: 7.081646
update!!!!!


...

Data Preparation: 5.8s
Epoch: 534 (44.5s)  LR: 0.00052 Train Loss: 3.021698  Valid Loss: 2.478319
Data Preparation: 5.0s
Epoch: 535 (46.9s)  LR: 0.00054 Train Loss: 3.115003  Valid Loss: 2.355825
Data Preparation: 6.5s
Epoch: 536 (48.6s)  LR: 0.00056 Train Loss: 3.056776  Valid Loss: 2.485155
Data Preparation: 5.8s
Epoch: 537 (44.4s)  LR: 0.00059 Train Loss: 3.116589  Valid Loss: 2.550787
Data Preparation: 5.5s
Epoch: 538 (41.0s)  LR: 0.00061 Train Loss: 2.936365  Valid Loss: 2.385603
Data Preparation: 4.3s
Epoch: 539 (44.1s)  LR: 0.00063 Train Loss: 3.079365  Valid Loss: 2.710664
Data Preparation: 4.9s
Epoch: 540 (49.7s)  LR: 0.00065 Train Loss: 3.041354  Valid Loss: 2.640465
Data Preparation: 4.4s
Epoch: 541 (48.0s)  LR: 0.00068 Train Loss: 3.108111  Valid Loss: 2.353098
Data Preparation: 4.9s
Epoch: 542 (45.2s)  LR: 0.00070 Train Loss: 3.009317  Valid Loss: 2.282035
update!!!!!


/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TransformerModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TransformerEncoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleList. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TransformerEncoderLayer. It won't be checked for correctness upon loadin

Data Preparation: 5.2s
Epoch: 543 (43.3s)  LR: 0.00072 Train Loss: 3.131003  Valid Loss: 2.483179
Data Preparation: 4.5s
Epoch: 544 (46.0s)  LR: 0.00074 Train Loss: 3.054753  Valid Loss: 2.410299
Data Preparation: 5.0s
Epoch: 545 (49.1s)  LR: 0.00076 Train Loss: 2.956861  Valid Loss: 2.609939
Data Preparation: 4.1s
Epoch: 546 (44.4s)  LR: 0.00078 Train Loss: 3.012630  Valid Loss: 2.609587
Data Preparation: 4.8s
Epoch: 547 (43.4s)  LR: 0.00080 Train Loss: 3.091207  Valid Loss: 2.491700
Data Preparation: 4.5s
Epoch: 548 (44.6s)  LR: 0.00082 Train Loss: 3.137368  Valid Loss: 2.694670
Data Preparation: 5.8s
Epoch: 549 (47.1s)  LR: 0.00084 Train Loss: 2.965690  Valid Loss: 2.705561
Data Preparation: 5.8s
Epoch: 550 (50.7s)  LR: 0.00085 Train Loss: 2.951108  Valid Loss: 2.470384
Data Preparation: 4.7s
Epoch: 551 (47.6s)  LR: 0.00087 Train Loss: 3.080766  Valid Loss: 2.591681
Data Preparation: 4.2s
Epoch: 552 (46.9s)  LR: 0.00089 Train Loss: 2.948183  Valid Loss: 2.451949
Data Preparation: 3.

/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TransformerModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TransformerEncoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ModuleList. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type TransformerEncoderLayer. It won't be checked for correctness upon loadin

Data Preparation: 5.5s
Epoch: 595 (49.9s)  LR: 0.00062 Train Loss: 2.963438  Valid Loss: 2.442980
Data Preparation: 4.5s
Epoch: 596 (46.3s)  LR: 0.00059 Train Loss: 2.981436  Valid Loss: 2.470928
Data Preparation: 4.8s
Epoch: 597 (46.6s)  LR: 0.00057 Train Loss: 2.997007  Valid Loss: 2.520121
Data Preparation: 4.7s
Epoch: 598 (48.3s)  LR: 0.00055 Train Loss: 2.954102  Valid Loss: 2.412135
Data Preparation: 4.8s


In [ ]:
with torch.no_grad():
        valid_accs   = []
        valid_losses = []
        for data, paper_ids, texts in valid_data:
            node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = data
            reps = gnn(node_feature.to(device), node_type.to(device), edge_time.to(device), \
                       edge_index.to(device), edge_type.to(device))[node_dict['paper'][0] + paper_ids]
            inp_idxs, out_idxs, masks, lens = parse_text(texts, w2v_model, device)
            pred_prob = rnn(inp_idxs, reps.repeat(inp_idxs.shape[0], 1, 1))
            loss = criterion(pred_prob[masks], out_idxs[masks])
            valid_loss = loss.cpu().detach().tolist()

            l_s = []
            s = 0
            for i in lens:
                l_s += [np.average(valid_loss[s: s+i])]
                s = s+i
            predict_sent = []
            label_sent   = []
            for s, text in zip(pred_prob.transpose(0,1).argmax(dim=-1), texts):
                sent = []
                for w in s:
                    word = w2v_model.wv.index2word[w]
                    sent += [w2v_model.wv.index2word[w]]
                    if word == '<eos>':
                        break
                label_sent += [text]
                predict_sent += [' '.join(sent)]

            for i in np.argsort(l_s):
                print('Loss: %.4f' % l_s[i])
                print('Predict: %s' % predict_sent[i])
                print('Label: %s' % label_sent[i])

            valid_losses += [np.average(valid_loss)]
            del loss, pred_prob
        st = time.time()
        print(("Epoch: %d (%.1fs)  LR: %.5f Train Loss: %f  Valid Loss: %f") % \
              (epoch, (st-et), optimizer.param_groups[0]['lr'], np.average(train_losses), \
               np.average(valid_losses)))
        if np.average(valid_losses) < best_val:
            best_val = np.average(valid_losses)
            torch.save(rnn, './save/rnn.pt')
            torch.save(gnn, './save/gen_gnn.pt')
        stats += [[train_losses, valid_losses]]

In [ ]:
losses = []
for epoch, (train_losses, valid_losses) in enumerate(stats):
    for ti in train_losses:
        losses += [[ti, 'Train', epoch]]
    for ti in valid_losses:
        losses += [[ti, 'Valid', epoch]]
sb.lineplot(data = pd.DataFrame(losses, columns=['loss', 'Type', 'Epoch']), x = 'Epoch', y='loss', hue='Type')
plt.show()

In [ ]:
loss

In [ ]:
valid_accs   = []
valid_losses = []
for data, paper_ids, texts in valid_data:
    node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = data
    reps = gnn(node_feature.to(device), node_type.to(device), edge_time.to(device), \
               edge_index.to(device), edge_type.to(device))[node_dict['paper'][0] + paper_ids]
    inp_idxs, out_idxs, masks, lens = parse_text(texts, w2v_model, device)
    pred_prob = rnn(inp_idxs, reps.repeat(inp_idxs.shape[0], 1, 1))
    loss = criterion(pred_prob[masks], out_idxs[masks])
    valid_loss = loss.cpu().detach().tolist()

    l_s = []
    s = 0
    for i in lens:
        l_s += [np.average(valid_loss[s: s+i])]
        s = s+i
    predict_sent = []
    label_sent   = []
    for s, text in zip(pred_prob.transpose(0,1).argmax(dim=-1), texts):
        sent = []
        for w in s:
            word = w2v_model.wv.index2word[w]
            sent += [w2v_model.wv.index2word[w]]
            if word == '<eos>':
                break
        label_sent += [text]
        predict_sent += [' '.join(sent)]

    for i in np.argsort(l_s):
        print('Loss: %.4f' % l_s[i])
        print('Predict: %s' % predict_sent[i])
        print('Label: %s' % label_sent[i])

    valid_losses += [np.average(valid_loss)]

In [17]:
edge_time.min()

tensor(45)

In [ ]:
ban   = defaultdict( #target_type
                        lambda: defaultdict(lambda: -100) # {target_id}
                    )

In [ ]:
ban['paper'][123] = 0

In [ ]:
ban['paper'][456]

In [14]:
for i in range(10):
    print(type(i))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [15]:
node_feature.to(device)

RuntimeError: CUDA error: device-side assert triggered

In [107]:
def sample_subgraph(graph, time_range, sampled_depth = 2, sampled_number = 8, inp = None):
    '''
        Sample Sub-Graph based on the connection of other nodes with currently sampled nodes
        We maintain budgets for each node type, indexed by <node_id, time>.
        Currently sampled nodes are stored in layer_data.
        After nodes are sampled, we construct the sampled adjacancy matrix.
    '''
    layer_data  = defaultdict( #target_type
                        lambda: {} # {target_id: [ser, time]}
                    )
    budget     = defaultdict( #source_type
                                    lambda: defaultdict(  #source_id
                                        lambda: [0., 0] #[sampled_score, time]
                            ))
    new_layer_adj  = defaultdict( #target_type
                                    lambda: defaultdict(  #source_type
                                        lambda: defaultdict(  #relation_type
                                            lambda: [] #[target_id, source_id]
                                )))
    '''
        For each node being sampled, we find out all its neighborhood, 
        adding the degree count of these nodes in the budget.
        Note that there exist some nodes that have many neighborhoods
        (such as fields, venues), for those case, we only consider 
    '''
    def add_budget(te, target_id, target_time, layer_data, budget):
        for source_type in te:
            tes = te[source_type]
            for relation_type in tes:
                if relation_type == 'self' or target_id not in tes[relation_type]:
                    continue
                adl = tes[relation_type][target_id]
                if len(adl) < sampled_number:
                    sampled_ids = list(adl.keys())
                else:
                    sampled_ids = np.random.choice(list(adl.keys()), sampled_number, replace = False)
                for source_id in sampled_ids:
                    source_time = adl[source_id]
                    if source_time == None:
                        source_time = target_time
                    if source_time not in time_range or source_id in layer_data[source_type]:
                        continue
                    budget[source_type][source_id][0] += 1. / len(sampled_ids)
                    budget[source_type][source_id][1] = source_time
    '''
        If inp == None: we sample some paper as initial nodes;
        else:           we are dealing with a specific supervised task (e.g. author disambiguation),
                        where some node pairs are given as output.
    '''
    
    if inp == None:
        _time = np.random.choice(list(time_range.keys()))
        res = graph.node_feature['paper'][graph.node_feature['paper']['time'] == _time]
        sampn = min(len(res), sampled_number)
        rand_paper_ids  = np.random.choice(list(res.index), sampn, replace = False)
        '''
            First adding the sampled nodes then updating budget.
        '''
        for _id in rand_paper_ids:
            layer_data['paper'][_id] = [len(layer_data['paper']), _time]
        for _id in rand_paper_ids:
            add_budget(graph.edge_list['paper'], _id, _time, layer_data, budget)
    else:
        '''
            First adding the sampled nodes then updating budget.
        '''
        for _type in inp:
            for _id, _time in inp[_type]:
                layer_data[_type][_id] = [len(layer_data[_type]), _time]
        for _type in inp:
            te = graph.edge_list[_type]
            for _id, _time in inp[_type]:
                add_budget(te, _id, _time, layer_data, budget)
    '''
        We recursively expand the sampled graph by sampled_depth.
        Each time we sample a fixed number of nodes for each budget,
        based on the accumulated degree.
    '''
    for layer in range(sampled_depth):
        sts = list(budget.keys())
        for source_type in sts:
            te = graph.edge_list[source_type]
            keys  = np.array(list(budget[source_type].keys()))
            if sampled_number > len(keys):
                '''
                    Directly sample all the nodes
                '''
                sampled_ids = np.arange(len(keys))
            else:
                '''
                    Sample based on accumulated degree
                '''
                score = np.array(list(budget[source_type].values()))[:,0] ** 2
                score = score / np.sum(score)
                sampled_ids = np.random.choice(len(score), sampled_number, p = score, replace = False) 
            sampled_keys = keys[sampled_ids]
            '''
                First adding the sampled nodes then updating budget.
            '''
            for k in sampled_keys:
                layer_data[source_type][k] = [len(layer_data[source_type]), budget[source_type][k][1]]
            for k in sampled_keys:
                add_budget(te, k, budget[source_type][k][1], layer_data, budget)
                budget[source_type].pop(k)   
    '''
        Prepare feature, time and adjacency matrix for the sampled graph
    '''
    feature = {}
    times   = {}
    indxs   = {}
    texts   = []
    for _type in layer_data:
        if len(layer_data[_type]) == 0:
            continue
        idxs  = np.array(list(layer_data[_type].keys()))
        tims  = np.array(list(layer_data[_type].values()))[:,0]
        
        if 'node_emb' in graph.node_feature[_type]:
            feature[_type] = np.array(list(graph.node_feature[_type].loc[idxs, 'node_emb']), dtype=np.float)
        else:
            feature[_type] = np.zeros([len(idxs), 400])
        feature[_type] = np.concatenate((feature[_type], list(graph.node_feature[_type].loc[idxs, 'emb']),\
            np.log10(np.array(list(graph.node_feature[_type].loc[idxs, 'citation'])).reshape(-1, 1) + 0.01)), axis=1)
        
        times[_type]   = tims
        indxs[_type]   = idxs
        
        if _type == 'paper':
            texts = np.array(list(graph.node_feature[_type].loc[idxs, 'title']), dtype=np.str)
    edge_list = defaultdict( #target_type
                        lambda: defaultdict(  #source_type
                            lambda: defaultdict(  #relation_type
                                lambda: [] # [target_id, source_id] 
                                    )))
    for _type in layer_data:
        for _key in layer_data[_type]:
            _ser = layer_data[_type][_key][0]
            edge_list[_type][_type]['self'] += [[_ser, _ser]]
    '''
        Reconstruct sampled adjacancy matrix by checking whether each
        link exist in the original graph
    '''
    for target_type in graph.edge_list:
        te = graph.edge_list[target_type]
        for source_type in te:
            tes = te[source_type]
            for relation_type in tes:
                tesr = tes[relation_type]
                for target_key in layer_data[target_type]:
                    target_ser = layer_data[target_type][target_key][0]
                    if target_key not in tesr:
                        continue
                    tesrt = tesr[target_key]
                    for source_key in layer_data[source_type]:
                        source_ser = layer_data[source_type][source_key][0]
                        '''
                            Check whether each link (target_id, source_id) exist in original adjacancy matrix
                        '''
                        if source_key in tesrt:
                            edge_list[target_type][source_type][relation_type] += [[target_ser, source_ser]]
    return feature, times, edge_list, indxs, texts

def to_torch(feature, time, edge_list, graph):
    '''
        Transform a sampled sub-graph into pytorch Tensor
        node_dict: {node_type: <node_number, node_type_ID>} node_number is used to trace back the nodes in original graph.
        edge_dict: {edge_type: edge_type_ID}
    '''
    node_dict = {}
    node_feature = []
    node_type    = []
    node_time    = []
    edge_index   = []
    edge_type    = []
    edge_time    = []
    
    node_num = 0
    types = graph.get_types()
    for t in types:
        node_dict[t] = [node_num, len(node_dict)]
        node_num     += len(feature[t])
    for t in types:
        node_feature += list(feature[t])
        node_time    += list(time[t])
        node_type    += [node_dict[t][1] for _ in range(len(feature[t]))]
        
    edge_dict = {e[2]: i for i, e in enumerate(graph.get_meta_graph())}
    edge_dict['self'] = len(edge_dict)
    
    for target_type in edge_list:
        for source_type in edge_list[target_type]:
            for relation_type in edge_list[target_type][source_type]:
                for ii, (ti, si) in enumerate(edge_list[target_type][source_type][relation_type]):
                    tid, sid = ti + node_dict[target_type][0], si + node_dict[source_type][0]
                    edge_index += [[tid, sid]]
                    edge_type  += [edge_dict[relation_type]]   
                    '''
                        Our time ranges from 1900 - 2020, largest span is 120.
                    '''
                    edge_time  += [node_time[tid] - node_time[sid] + 120]
    node_feature = torch.FloatTensor(node_feature)
    node_type    = torch.LongTensor(node_type)
    edge_time    = torch.LongTensor(edge_time)
    edge_index   = torch.LongTensor(edge_index).t()
    edge_type    = torch.LongTensor(edge_type)
    return node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict

In [111]:
stats = []
pool = mp.Pool(1)
process_ids = np.arange(2)
st = time.time()
jobs = prepare_data(pool, process_ids)
train_step = 1500
best_val   = 100000
criterion = nn.CrossEntropyLoss(reduction = 'none')

defaultdict(<function sample_subgraph.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda> at 0x7f1329bf57b8>, {'rev_PF_in_L0': [[0, 1], [1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1], [8, 1], [9, 1], [10, 1], [11, 1], [12, 1], [13, 1], [14, 1], [15, 1], [16, 1], [17, 1], [18, 1], [19, 1], [20, 1], [21, 1], [22, 1], [23, 1], [24, 1], [25, 1], [26, 1], [27, 1], [28, 1], [29, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1], [35, 1], [36, 1], [37, 1], [38, 1], [39, 1], [40, 1], [41, 1], [42, 1], [43, 1], [44, 1], [45, 1], [46, 1], [47, 1], [48, 1], [49, 1], [50, 1], [51, 1], [52, 1], [53, 1], [54, 1], [55, 1], [56, 1], [57, 1], [58, 1], [59, 1], [60, 1], [61, 1], [62, 1], [63, 1], [64, 1], [65, 1], [66, 1], [67, 1], [68, 1], [69, 1], [70, 1], [71, 1], [72, 1], [73, 1], [74, 1], [75, 1], [76, 1], [77, 1], [78, 1], [79, 1], [80, 1], [81, 1], [82, 1], [83, 1], [84, 1], [85, 1], [86, 1], [87, 1], [88, 1], [89, 1], [90, 1], [91, 1], [92, 1], [93, 1], [94, 1], [95, 1], [96, 1], [97, 1]

defaultdict(<function sample_subgraph.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda> at 0x7f1329bf57b8>, {'rev_PF_in_L0': [[0, 1], [1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1], [8, 1], [9, 1], [10, 1], [11, 1], [12, 1], [13, 1], [14, 1], [15, 1], [16, 1], [17, 1], [18, 1], [19, 1], [20, 1], [21, 1], [22, 1], [23, 1], [24, 1], [25, 1], [26, 1], [27, 1], [28, 1], [29, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1], [35, 1], [36, 1], [37, 1], [38, 1], [39, 1], [40, 1], [41, 1], [42, 1], [43, 1], [44, 1], [45, 1], [46, 1], [47, 1], [48, 1], [49, 1], [50, 1], [51, 1], [52, 1], [53, 1], [54, 1], [55, 1], [56, 1], [57, 1], [58, 1], [59, 1], [60, 1], [61, 1], [62, 1], [63, 1], [64, 1], [65, 1], [66, 1], [67, 1], [68, 1], [69, 1], [70, 1], [71, 1], [72, 1], [73, 1], [74, 1], [75, 1], [76, 1], [77, 1], [78, 1], [79, 1], [80, 1], [81, 1], [82, 1], [83, 1], [84, 1], [85, 1], [86, 1], [87, 1], [88, 1], [89, 1], [90, 1], [91, 1], [92, 1], [93, 1], [94, 1], [95, 1], [96, 1], [97, 1]

26 211 paper paper rev_PP_cite 32
71 116 paper paper rev_PP_cite 76
80 114 paper paper rev_PP_cite 85
86 121 paper paper rev_PP_cite 90
89 120 paper paper rev_PP_cite 93
114 80 paper paper rev_PP_cite 113
116 71 paper paper rev_PP_cite 114
120 89 paper paper rev_PP_cite 118
121 86 paper paper rev_PP_cite 119
158 261 paper paper rev_PP_cite 166
168 206 paper paper rev_PP_cite 177
206 168 paper paper rev_PP_cite 218
206 405 paper paper rev_PP_cite 219
211 26 paper paper rev_PP_cite 223
261 158 paper paper rev_PP_cite 262
288 321 paper paper rev_PP_cite 286
321 288 paper paper rev_PP_cite 307
405 206 paper paper rev_PP_cite 363


In [113]:
train_data = [job.get() for job in jobs]
# valid_data = [jobs[-2].get(), jobs[-1].get()]
pool.close()
pool.join()

In [ ]:
node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = data
reps = gnn(node_feature.to(device), node_type.to(device), edge_time.to(device), \
           edge_index.to(device), edge_type.to(device))[node_dict['paper'][0] + paper_ids]

In [141]:
edge_index[1][edge_index[0] == 295]

tensor([ 380,  103,  505,  606,  803, 1015,  806,  811,  818,  913, 1197, 1248,
        1284, 1402, 1345])